In [38]:
%env BEARER_TOKEN=AAAAAAAAAAAAAAAAAAAAAA%2B4dAEAAAAAOZaAcisubt%2FTYUkSvYkFuNzs%2B0w%3DX7y2B0PLyb8Bdju9DeA9Z1vuzgaqZHPPj3cunjGhV6NtP2G7UT
%env DEEPL_AUTH_KEY=OUO
%env MICROSOFTTRANSLATOR_KEY=69897d6f6dda4b49aa07aee9effd4d10

env: BEARER_TOKEN=AAAAAAAAAAAAAAAAAAAAAA%2B4dAEAAAAAOZaAcisubt%2FTYUkSvYkFuNzs%2B0w%3DX7y2B0PLyb8Bdju9DeA9Z1vuzgaqZHPPj3cunjGhV6NtP2G7UT
env: DEEPL_AUTH_KEY=OUO
env: MICROSOFTTRANSLATOR_KEY=69897d6f6dda4b49aa07aee9effd4d10


In [49]:
import os
import requests
import json
import time
bearer_token = os.environ.get("BEARER_TOKEN")
deepl_auth_key = os.environ.get("DEEPL_AUTH_KEY")
microsofttranslator_key = os.environ.get("MICROSOFTTRANSLATOR_KEY")

In [12]:
usernames = [
    "", # 0
    "", # 1
    "kishida230", # 岸田文雄
    "matsunohirokazu", # 松野博一
    "nekotanchan", # 金子恭之
    "hayashi09615064", # 林芳正
    "_suzukitakako_", # 铃木贵子
    "SSS_suematsu", # 末松信介
    "goto_shigeyuki", # 后藤茂之
    "", # 金子原二郎
    "saitotetsuo", # 齐藤铁夫
    "KishiNobuo", # 岸信夫
    "", # 山东昭子
    "hosodahiroyuki", # 细田博之
    "", # 铃木俊一
    "yamagiwa001", # 山际大志郎
    "", # 萩生田光一
    "244yamaguchi", # 山口壮
    "ninoyu_satoshi", # 二之汤智
    "NishimeKosaburo", # 西铭恒三郎
    "makishimakaren", # 牧岛花莲
    "noda_seiko93", # 野田圣子
    "wakamiya7788", # 若宫健嗣
    "kobahawk" # 小林鹰之
]

In [13]:
def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    return r
def retrieve_user_with_username(username):
    response = requests.get(url=f'https://api.twitter.com/2/users/by/username/{username}', auth=bearer_oauth)
    result = json.loads(response.text)
    return result['data']['id']
# id = retrieve_user_with_username('kishida230')

In [15]:
def get_tweets(id):
    tweets = []
    params = {
        "exclude": "retweets",
        "max_results": 100,
        "tweet.fields": "created_at"
    }
    response = requests.get(url=f'https://api.twitter.com/2/users/{id}/tweets', params=params, auth=bearer_oauth)
    result = json.loads(response.text)
    tweets += result['data']
    while 'next_token' in result['meta']:
        next_token = result['meta']['next_token']
        params = {
            "exclude": "retweets",
            "max_results": 100,
            "tweet.fields": "created_at",
            "pagination_token": next_token
        }
        response = requests.get(url=f'https://api.twitter.com/2/users/{id}/tweets', params=params, auth=bearer_oauth)
        result = json.loads(response.text)
        tweets += result['data']
    return tweets
# tweets = get_tweets('1247032696586436609')

In [17]:
def deepl_oauth(r):
    r.headers["Authorization"] = f"DeepL-Auth-Key {deepl_auth_key}"
    return r
def translator(text):
    # response = requests.get(url='https://api.deepl.com/v2/usage', auth=deepl_oauth)
    # print(response.text)
    data = {
        "text": text,
        "target_lang": "ZH"
    }
    response = requests.get(url='https://api.deepl.com/v2/translate', auth=deepl_oauth, data=data)
    result = json.loads(response.text)
    return result['translations'][0]['text']
# translator('test')
def translator_usage():
    response = requests.get(url='https://api.deepl.com/v2/usage', auth=deepl_oauth)
    print(response.text)

In [47]:
import requests, uuid, json

def microsoft_translator(text_list):
    endpoint = "https://api.cognitive.microsofttranslator.com"
    location = "eastasia"
    path = '/translate'
    constructed_url = endpoint + path
    params = {
        'api-version': '3.0',
        'to': 'zh-Hans'
    }
    headers = {
        'Ocp-Apim-Subscription-Key': microsofttranslator_key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }
    body = []
    for text in text_list:
        body.append({
            'text': text
        })
    response = requests.post(constructed_url, params=params, headers=headers, json=body)
    result = json.loads(response.text)
    # print(result)
    translated_text_list = []
    for item in result:
        translated_text_list.append(item['translations'][0]['text'])
    return translated_text_list
# print(microsoft_translator(['test1', 'test2']))

In [42]:
import pickle
for username in usernames:
    if username == "":
        continue
    print(username)
    id = retrieve_user_with_username(username)
    tweets = get_tweets(id)
    # print(tweets)
    tweets_pk_path = f"{username}.pk"
    tweets_pk = open(tweets_pk_path, 'wb')
    pickle.dump(tweets, tweets_pk)
    tweets_pk.close()

kishida230
matsunohirokazu
nekotanchan
hayashi09615064
_suzukitakako_
SSS_suematsu
goto_shigeyuki
saitotetsuo
KishiNobuo
hosodahiroyuki
yamagiwa001
244yamaguchi
ninoyu_satoshi
NishimeKosaburo
makishimakaren
noda_seiko93
wakamiya7788
kobahawk


In [56]:
for username in usernames:
    if username != "kishida230":
        continue
    print(username)
    tweets_pk_path = f"{username}.pk"
    tweets_pk = open(tweets_pk_path, 'rb')
    tweets = pickle.load(tweets_pk)
    tweets_pk.close()
    print(len(tweets))
    num_tweets = len(tweets)
    text_cnt = 0
    text_list = []
    translate_cnt = 0
    for i in range(0, num_tweets):
        text = tweets[i]['text']
        text_list.append(text)
        text_cnt += 1
        if text_cnt == 100:
            translated_text_list = microsoft_translator(text_list)
            translate_cnt += 1
            print(translate_cnt)
            for j in range(0, text_cnt):
                tweets[i + 1 - (text_cnt - j)]['chinese_content'] = translated_text_list[j]
            text_cnt = 0
            text_list = []
    if text_cnt != 0:
        translated_text_list = microsoft_translator(text_list)
        translate_cnt += 1
        print(translate_cnt)
        for j in range(0, text_cnt):
            tweets[num_tweets - (text_cnt - j)]['chinese_content'] = translated_text_list[j]
            # print(num_tweets - (text_cnt - j), j)
    tweets_pk_path = f"{username}_with_translation.pk"
    tweets_pk = open(tweets_pk_path, 'wb')
    pickle.dump(tweets, tweets_pk)
    tweets_pk.close()
    time.sleep(translate_cnt)

kishida230
980
1
2
3
4
5
6
7
8
9
10
